<a href="https://colab.research.google.com/github/javihm77/Data-Science/blob/master/MNIST_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Import dependencies
import torch
from PIL import Image
from torch import nn, save, load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [6]:
# Get data
train = datasets.MNIST(root="data", download=True, train=True, transform=ToTensor())
dataset = DataLoader(train, 32)
#1,28,28 - classes 0-9

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Image Classifier Neural Network
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, (3,3)),
            nn.ReLU(),
            nn.Conv2d(32, 64, (3,3)),
            nn.ReLU(),
            nn.Conv2d(64, 64, (3,3)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*(28-6)*(28-6), 10)
        )

    def forward(self, x):
        return self.model(x)

In [9]:
# Instance of the neural network, loss, optimizer

clf = ImageClassifier().to('cuda')
#clf = ImageClassifier()
opt = Adam(clf.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

In [10]:
# Training flow
if __name__ == "__main__":
    for epoch in range(10): # train for 10 epochs
        for batch in dataset:
            X,y = batch
            X, y = X.to('cuda'), y.to('cuda')
            yhat = clf(X)
            loss = loss_fn(yhat, y)

            # Apply backprop
            opt.zero_grad()
            loss.backward()
            opt.step()

        print(f"Epoch:{epoch} loss is {loss.item()}")

    with open('model_state.pt', 'wb') as f:
        save(clf.state_dict(), f)

    with open('model_state.pt', 'rb') as f:
        clf.load_state_dict(load(f))


Epoch:0 loss is 0.015325346030294895
Epoch:1 loss is 0.009968436323106289
Epoch:2 loss is 0.004126145038753748
Epoch:3 loss is 0.0001321040908806026
Epoch:4 loss is 4.526141128735617e-05
Epoch:5 loss is 3.226041599191376e-06
Epoch:6 loss is 2.1832658603671007e-05
Epoch:7 loss is 3.0211576813599095e-06
Epoch:8 loss is 5.178129640626139e-07
Epoch:9 loss is 3.7252892326478104e-08


<ipython-input-10-d4ef3289aee7>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clf.load_state_dict(load(f))


In [16]:
img = Image.open('img_1.jpg')

img_tensor = ToTensor()(img).unsqueeze(0).to('cuda')

print(torch.argmax(clf(img_tensor)))

AttributeError: type object 'Image' has no attribute 'open'

In [12]:
img = Image.open('img_2.jpg')
img_tensor = ToTensor()(img).unsqueeze(0).to('cuda')

print(torch.argmax(clf(img_tensor)))

tensor(0, device='cuda:0')


In [13]:
img = Image.open('img_3.jpg')
img_tensor = ToTensor()(img).unsqueeze(0).to('cuda')

print(torch.argmax(clf(img_tensor)))

tensor(9, device='cuda:0')


In [17]:
from IPython.display import display, Image

# img_1.jpg
image_path = 'img_1.jpg'
# Display the image
display(Image(filename=image_path))

image_path = 'img_2.jpg'
# Display the image
display(Image(filename=image_path))

image_path = 'img_3.jpg'
# Display the image
display(Image(filename=image_path))


In [18]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Load MNIST test dataset
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(test_data, batch_size=32)

# Assuming 'clf' is your trained model
clf.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculations during testing
    for images, labels in test_loader:
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = clf(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the 10000 test images: {accuracy:.2f}%')

Accuracy of the model on the 10000 test images: 98.83%
